In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Environment Setup
!pip install timm lpips opencv-python

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 5.7 MB/s eta 0:00:00


In [ ]:
import os

base_path = f"/content/drive/MyDrive/"

print(f"Checking paths in: {base_path} ...")
if os.path.exists(base_path):
    print("✅ Project Folder Found!")
    print("Contents:", os.listdir(base_path))
else:
    print(f"❌ Error: Folder not found.")

Checking paths in: /content/drive/MyDrive/ ...
✅ Project Folder Found!
Contents: ['Narendra_Avasthi_Latest_14e_Problems_in_Physicsal_Chemistry_Book.pdf', 'Generated PDF from Image Files.pdf', 'FCP Laboratory Journal-1 (Final).docx', 'EC-C U24EV040 HITESH BEHANI.gdoc', 'Table of Contents.pdf', 'bookswap', 'bookswap 2', '173761784656690659225098794412.jpg', 'Untitled document.gdoc', '17432274405556073219855521790897.jpg', 'Presentation 15 (2).pptx', '174720725515982815960953652529.jpg', 'Colab Notebooks', 'Task 2.docx', 'ecell.pptx', 'Mrudul-Mittal-FlowCV-Resume-20250805.pdf', 'Mrudul-Mittal-FlowCV-Resume-20250805 (1).pdf', 'PEEBM FINAL.gdoc', 'LIST OF TABLES.gdoc', 'peebm report.gdoc', 'aadharcard.pdf', 'Delhi!!!', 'Shadow removal', 'Colab_26dB_Results', 'genesis', 'Recording_20260113_232609.mp4', 'Sequence 01_7.mp4']


In [ ]:
# 1. Mount Drive (Data & Weights ke liye)
from google.colab import drive
drive.mount('/content/drive')

import os
%cd /content
!git clone https://github.com/Yuyok/FusionShadowRemoval-main
REPO_NAME = "FusionShadowRemoval-main"
%cd /content/{REPO_NAME}

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content
Cloning into 'FusionShadowRemoval-main'...
remote: Enumerating objects: 116, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 116 (delta 2), reused 116 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (116/116), 3.97 MiB | 4.48 MiB/s, done.
Resolving deltas: 100% (2/2), done.
/content/FusionShadowRemoval-main


In [ ]:
!pip install kornia pytorch-lightning timm==0.6.13 lpips opencv-python pyyaml omegaconf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 kB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.5/849.5 kB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 128.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 69.7 MB/s eta 0:00:00
  Attempting uninstall: timm
    Found existing installation: timm 1.0.24
    Uninstalling timm-1.0.24:
      Successfully uninstalled timm-1.0.24


In [ ]:
import sys
import os

# Path set karo
%cd /content/FusionShadowRemoval-main
sys.path.append(os.getcwd())

# Direct import try karo (Agar ye fail hua to asli wajah dikhegi)
from models.model_convnext import fusion_net
print("✅ Agar ye print hua, toh model import ho gaya!")

/content/FusionShadowRemoval-main
✅ Agar ye print hua, toh model import ho gaya!


basic loss function not the one used in fusion net

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import os
import glob
import cv2
import numpy as np
import random
import sys
import shutil

# --- 1. SETUP PATHS ---
%cd /content/FusionShadowRemoval-main
sys.path.append(os.getcwd())

DRIVE_BASE = "/content/drive/MyDrive/Shadow removal"
LOCAL_BASE = "/content/dataset"

# --- 2. FORCE COPY DATA (FIXED) ---
def force_copy_data():
    print("🔄 Checking Data Integrity...")

    # Agar purana folder hai, toh usko uda do taaki fresh copy ho sake
    if os.path.exists(LOCAL_BASE):
        print("🗑️ Removing old/incomplete local data...")
        shutil.rmtree(LOCAL_BASE)

    print("⏳ Copying data from Drive to Colab Local Disk...")
    os.makedirs(LOCAL_BASE, exist_ok=True)

    folders = [
        "ntire24_shrem_train_inp", "ntire24_shrem_train_gt",
        "ntire24_shrem_valid_inp", "ntire24_shrem_valid_gt"
    ]

    for folder in folders:
        src = os.path.join(DRIVE_BASE, folder)
        dst = os.path.join(LOCAL_BASE, folder)

        if os.path.exists(src):
            # Check source file count
            num_files = len(os.listdir(src))
            print(f"   📂 Copying {folder} ({num_files} images)...")
            shutil.copytree(src, dst)
        else:
            print(f"❌ Error: Folder '{folder}' Drive pe nahi mila! Check path: {src}")

    print("✅ Data Copy Complete! Training should start now.")

# Call the function
force_copy_data()

# --- 3. IMPORT MODEL ---
try:
    from models.model_convnext import fusion_net
except ImportError as e:
    print(f"❌ Model Import Error: {e}")
    sys.exit(1)

# --- 4. CONFIGURATION ---
EPOCHS = 5
BATCH_SIZE = 1
PATCH_SIZE = 256
LR = 2e-4

class ShadowDataset(Dataset):
    def __init__(self, inp_dir, gt_dir, is_train=True):
        self.inp_files = sorted(glob.glob(os.path.join(inp_dir, "*")))
        self.gt_dir = gt_dir
        self.is_train = is_train

        # Valid pairs check
        self.valid_files = [f for f in self.inp_files if os.path.exists(os.path.join(gt_dir, os.path.basename(f)))]

        # DEBUG PRINT
        if len(self.valid_files) == 0:
            print(f"❌ CRITICAL ERROR: No images found in {inp_dir}")
            print(f"   Checking GT dir: {gt_dir}")
        else:
            print(f"[{'TRAIN' if is_train else 'VALID'}] Loaded {len(self.valid_files)} images.")

    def __len__(self):
        return len(self.valid_files)

    def __getitem__(self, idx):
        inp_path = self.valid_files[idx]
        fname = os.path.basename(inp_path)
        gt_path = os.path.join(self.gt_dir, fname)

        img_inp = cv2.imread(inp_path)
        img_gt = cv2.imread(gt_path)

        if self.is_train:
            h, w = img_inp.shape[:2]
            if h > PATCH_SIZE and w > PATCH_SIZE:
                x = random.randint(0, w - PATCH_SIZE)
                y = random.randint(0, h - PATCH_SIZE)
                img_inp = img_inp[y:y+PATCH_SIZE, x:x+PATCH_SIZE]
                img_gt = img_gt[y:y+PATCH_SIZE, x:x+PATCH_SIZE]
            else:
                img_inp = cv2.resize(img_inp, (PATCH_SIZE, PATCH_SIZE))
                img_gt = cv2.resize(img_gt, (PATCH_SIZE, PATCH_SIZE))
        else:
            h, w = img_inp.shape[:2]
            new_h = ((h // 32) * 32)
            new_w = ((w // 32) * 32)
            if new_h != h or new_w != w:
                img_inp = cv2.resize(img_inp, (new_w, new_h))
                img_gt = cv2.resize(img_gt, (new_w, new_h))

        inp_t = torch.from_numpy(img_inp[:, :, ::-1].copy()).permute(2, 0, 1).float() / 255.0
        gt_t = torch.from_numpy(img_gt[:, :, ::-1].copy()).permute(2, 0, 1).float() / 255.0
        return inp_t, gt_t

def train():
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print(f"🚀 Training Started on {device}")

    # Initialize Datasets
    train_ds = ShadowDataset(
        inp_dir=os.path.join(LOCAL_BASE, "ntire24_shrem_train_inp"),
        gt_dir=os.path.join(LOCAL_BASE, "ntire24_shrem_train_gt"),
        is_train=True
    )

    # Check if dataset is empty before creating loader
    if len(train_ds) == 0:
        print("❌ Training Dataset is EMPTY. Stopping.")
        return

    val_ds = ShadowDataset(
        inp_dir=os.path.join(LOCAL_BASE, "ntire24_shrem_valid_inp"),
        gt_dir=os.path.join(LOCAL_BASE, "ntire24_shrem_valid_gt"),
        is_train=False
    )

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=2)

    model = fusion_net().to(device)
    optimizer = optim.AdamW(model.parameters(), lr=LR)
    criterion = nn.L1Loss()

    pkl_path = os.path.join(DRIVE_BASE, "shadowremoval.pkl")
    if os.path.exists(pkl_path):
        model.load_state_dict(torch.load(pkl_path, map_location=device), strict=False)
        print("✅ Pre-trained weights loaded!")

    for epoch in range(EPOCHS):
        model.train()
        train_loss = 0

        print(f"\n--- Epoch {epoch+1}/{EPOCHS} ---")
        for i, (inp, gt) in enumerate(train_loader):
            inp, gt = inp.to(device), gt.to(device)

            optimizer.zero_grad()
            out = model(inp)
            loss = criterion(out, gt)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            if i % 20 == 0:
                print(f"Iter {i}: Loss {loss.item():.4f}")

        # Validation
        model.eval()
        psnr_list = []
        with torch.no_grad():
            for inp, gt in val_loader:
                inp, gt = inp.to(device), gt.to(device)
                out = model(inp)
                mse = torch.mean((out - gt) ** 2).item()
                psnr = 100 if mse == 0 else 20 * np.log10(1.0 / np.sqrt(mse))
                psnr_list.append(psnr)

        avg_psnr = np.mean(psnr_list)
        print(f"🔴 EPOCH {epoch+1} FINISHED. Val PSNR: {avg_psnr:.4f} dB")

        save_path = os.path.join(DRIVE_BASE, f"model_epoch_{epoch+1}.pth")
        torch.save(model.state_dict(), save_path)
        print(f"💾 Saved: {save_path}")

if __name__ == "__main__":
    train()

In [ ]:
!pip install kornia lpips piq timm==0.6.13

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.9/106.9 kB 11.9 MB/s eta 0:00:00


correct loss function

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler
import os
import glob
import cv2
import numpy as np
import random
import sys
import lpips
from piq import MultiScaleSSIMLoss

# --- 1. SETUP PATHS ---
%cd /content/FusionShadowRemoval-main
sys.path.append(os.getcwd())

DRIVE_BASE = "/content/drive/MyDrive/Shadow removal"
LOCAL_BASE = "/content/dataset"

# --- IMPORT MODEL ---
try:
    from models.model_convnext import fusion_net
    print("✅ Model imported!")
except ImportError:
    sys.exit(1)

# --- 2. ADVANCED LOSSES ---
class FocalFrequencyLoss(nn.Module):
    def __init__(self, loss_weight=1.0, alpha=1.0):
        super().__init__()
        self.loss_weight = loss_weight
        self.alpha = alpha
    def forward(self, pred, target):
        return self.loss_weight * torch.mean(torch.abs(torch.fft.rfftn(pred) - torch.fft.rfftn(target)))

# --- 3. CONFIGURATION ---
EPOCHS = 5
BATCH_SIZE = 4
PATCH_SIZE = 256
LR = 1e-5

# Dataset Class
class ShadowDataset(Dataset):
    def __init__(self, inp_dir, gt_dir, is_train=True):
        self.inp_files = sorted(glob.glob(os.path.join(inp_dir, "*")))
        self.gt_dir = gt_dir
        self.is_train = is_train
        self.valid_files = [f for f in self.inp_files if os.path.exists(os.path.join(gt_dir, os.path.basename(f)))]

    def __len__(self): return len(self.valid_files)

    def __getitem__(self, idx):
        inp_path = self.valid_files[idx]
        fname = os.path.basename(inp_path)
        gt_path = os.path.join(self.gt_dir, fname)
        img_inp = cv2.imread(inp_path)
        img_gt = cv2.imread(gt_path)

        if self.is_train:
            h, w = img_inp.shape[:2]
            if h > PATCH_SIZE and w > PATCH_SIZE:
                x = random.randint(0, w - PATCH_SIZE); y = random.randint(0, h - PATCH_SIZE)
                img_inp = img_inp[y:y+PATCH_SIZE, x:x+PATCH_SIZE]
                img_gt = img_gt[y:y+PATCH_SIZE, x:x+PATCH_SIZE]
            else:
                img_inp = cv2.resize(img_inp, (PATCH_SIZE, PATCH_SIZE))
                img_gt = cv2.resize(img_gt, (PATCH_SIZE, PATCH_SIZE))
        else:
            h, w = img_inp.shape[:2]
            new_h, new_w = ((h // 32) * 32), ((w // 32) * 32)
            img_inp = cv2.resize(img_inp, (new_w, new_h))
            img_gt = cv2.resize(img_gt, (new_w, new_h))

        inp_t = torch.from_numpy(img_inp[:, :, ::-1].copy()).permute(2, 0, 1).float() / 255.0
        gt_t = torch.from_numpy(img_gt[:, :, ::-1].copy()).permute(2, 0, 1).float() / 255.0
        return inp_t, gt_t

def train():
    device = torch.device('cuda:0')
    print(f"🚀 Training with Advanced Losses on {device}")

    # Check Local Data
    if not os.path.exists(LOCAL_BASE):
        print("❌ Local Data not found! Please run the 'Copy Data' script first.")
        return

    train_ds = ShadowDataset(os.path.join(LOCAL_BASE, "ntire24_shrem_train_inp"), os.path.join(LOCAL_BASE, "ntire24_shrem_train_gt"), True)
    val_ds = ShadowDataset(os.path.join(LOCAL_BASE, "ntire24_shrem_valid_inp"), os.path.join(LOCAL_BASE, "ntire24_shrem_valid_gt"), False)

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=2)

    model = fusion_net().to(device)

    # Initialize Losses
    l1_crit = nn.L1Loss().to(device)
    lpips_crit = lpips.LPIPS(net='vgg').to(device)
    fft_crit = FocalFrequencyLoss(loss_weight=0.1, alpha=1.0).to(device)
    msssim_crit = MultiScaleSSIMLoss(data_range=1.0).to(device)

    optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=1e-8)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, EPOCHS, eta_min=5e-6)

    # Load Weights
    pkl_path = os.path.join(DRIVE_BASE, "shadowremoval.pkl")
    if os.path.exists(pkl_path):
        model.load_state_dict(torch.load(pkl_path, map_location=device), strict=False)
        print("✅ Pre-trained weights loaded!")

    for epoch in range(EPOCHS):
        model.train()
        total_loss = 0

        print(f"\n--- Epoch {epoch+1}/{EPOCHS} ---")
        for i, (inp, gt) in enumerate(train_loader):
            inp, gt = inp.to(device), gt.to(device)

            optimizer.zero_grad()

            # Forward
            out = model(inp)

            # --- THE FIX: CLAMPING ---
            # SSIM aur LPIPS ke liye output 0-1 ke beech hona zaroori hai
            out_clamped = torch.clamp(out, 0, 1)

            # 1. L1 Loss (Raw output pe, taaki model negative dena band kare)
            l1 = l1_crit(out, gt)

            # 2. Strict Losses (Clamped output pe)
            perceptual = lpips_crit(out_clamped, gt).mean()
            fft = fft_crit(out_clamped, gt)
            ms_ssim = 1 - msssim_crit(out_clamped, gt)

            # Weighted Sum
            loss = l1 + (0.05 * perceptual) + (0.1 * fft) + (0.2 * ms_ssim)

            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            if i % 20 == 0:
                print(f"Iter {i}: Loss {loss.item():.4f}")

        scheduler.step()

        # Validation
        model.eval()
        psnr_list = []
        with torch.no_grad():
            for inp, gt in val_loader:
                inp, gt = inp.to(device), gt.to(device)
                out = model(inp)
                # Validation mein bhi clamp kar dete hain safe side ke liye
                out = torch.clamp(out, 0, 1)

                mse = torch.mean((out - gt) ** 2).item()
                psnr = 100 if mse == 0 else 20 * np.log10(1.0 / np.sqrt(mse))
                psnr_list.append(psnr)

        avg_psnr = np.mean(psnr_list)
        print(f"🔴 EPOCH {epoch+1} DONE. Val PSNR: {avg_psnr:.4f} dB")

        save_path = os.path.join(DRIVE_BASE, f"model_pro_epoch_{epoch+1}.pth")
        torch.save(model.state_dict(), save_path)
        print(f"💾 Saved: {save_path}")

if __name__ == "__main__":
    train()

minor changes in loss function

In [ ]:
!pip install --upgrade cython numpy setuptools wheel

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler
import os
import glob
import cv2
import numpy as np
import random
import sys
import shutil
import lpips
from piq import MultiScaleSSIMLoss

# --- 1. SETUP PATHS ---
%cd /content/FusionShadowRemoval-main
sys.path.append(os.getcwd())

DRIVE_BASE = "/content/drive/MyDrive/Shadow removal"
LOCAL_BASE = "/content/dataset"

# --- 2. FORCE DATA REPAIR (THE FIX) ---
def force_fix_data():
    print("🔄 Checking Dataset Health...")

    # Check if data exists AND is not empty
    inp_path = os.path.join(LOCAL_BASE, "ntire24_shrem_train_inp")

    # Agar folder nahi hai YA folder khaali hai -> Re-copy karo
    if not os.path.exists(inp_path) or len(os.listdir(inp_path)) == 0:
        print("⚠️ Data missing or empty! Re-copying from Drive...")

        # Purana kachra saaf karo
        if os.path.exists(LOCAL_BASE):
            shutil.rmtree(LOCAL_BASE)

        # Fresh Copy
        os.makedirs(LOCAL_BASE, exist_ok=True)
        for folder in ["ntire24_shrem_train_inp", "ntire24_shrem_train_gt", "ntire24_shrem_valid_inp", "ntire24_shrem_valid_gt"]:
            src = os.path.join(DRIVE_BASE, folder)
            dst = os.path.join(LOCAL_BASE, folder)
            if os.path.exists(src):
                print(f"   📂 Copying {folder}...")
                shutil.copytree(src, dst)
            else:
                print(f"❌ Error: {folder} Drive pe nahi mila!")
        print("✅ Data Restoration Complete!")
    else:
        print("✅ Data is present and ready.")

force_fix_data()

# --- IMPORT MODEL ---
try:
    from models.model_convnext import fusion_net
    print("✅ Model imported!")
except ImportError:
    print("❌ Error: Model import fail hua.")
    sys.exit(1)

# --- 3. ADVANCED LOSSES ---
class FocalFrequencyLoss(nn.Module):
    def __init__(self, loss_weight=1.0, alpha=1.0):
        super().__init__()
        self.loss_weight = loss_weight
        self.alpha = alpha
    def forward(self, pred, target):
        return self.loss_weight * torch.mean(torch.abs(torch.fft.rfftn(pred) - torch.fft.rfftn(target)))

# --- 4. CONFIGURATION ---
EPOCHS = 15
BATCH_SIZE = 4
PATCH_SIZE = 256
LR = 5e-6

# Dataset Class
class ShadowDataset(Dataset):
    def __init__(self, inp_dir, gt_dir, is_train=True):
        self.inp_files = sorted(glob.glob(os.path.join(inp_dir, "*")))
        self.gt_dir = gt_dir
        self.is_train = is_train
        self.valid_files = [f for f in self.inp_files if os.path.exists(os.path.join(gt_dir, os.path.basename(f)))]

    def __len__(self): return len(self.valid_files)

    def __getitem__(self, idx):
        inp_path = self.valid_files[idx]
        fname = os.path.basename(inp_path)
        gt_path = os.path.join(self.gt_dir, fname)
        img_inp = cv2.imread(inp_path)
        img_gt = cv2.imread(gt_path)

        if self.is_train:
            h, w = img_inp.shape[:2]
            if h > PATCH_SIZE and w > PATCH_SIZE:
                x = random.randint(0, w - PATCH_SIZE); y = random.randint(0, h - PATCH_SIZE)
                img_inp = img_inp[y:y+PATCH_SIZE, x:x+PATCH_SIZE]
                img_gt = img_gt[y:y+PATCH_SIZE, x:x+PATCH_SIZE]
            else:
                img_inp = cv2.resize(img_inp, (PATCH_SIZE, PATCH_SIZE))
                img_gt = cv2.resize(img_gt, (PATCH_SIZE, PATCH_SIZE))
        else:
            h, w = img_inp.shape[:2]
            new_h, new_w = ((h // 32) * 32), ((w // 32) * 32)
            img_inp = cv2.resize(img_inp, (new_w, new_h))
            img_gt = cv2.resize(img_gt, (new_w, new_h))

        inp_t = torch.from_numpy(img_inp[:, :, ::-1].copy()).permute(2, 0, 1).float() / 255.0
        gt_t = torch.from_numpy(img_gt[:, :, ::-1].copy()).permute(2, 0, 1).float() / 255.0
        return inp_t, gt_t

def train():
    device = torch.device('cuda:0')
    print(f"🚀 Fine-Tuning Started on {device} (Target: 27+ dB)")

    train_ds = ShadowDataset(os.path.join(LOCAL_BASE, "ntire24_shrem_train_inp"), os.path.join(LOCAL_BASE, "ntire24_shrem_train_gt"), True)

    # Final Safety Check
    if len(train_ds) == 0:
        print("❌ CRITICAL: Dataset is still empty! Check Drive paths.")
        return

    val_ds = ShadowDataset(os.path.join(LOCAL_BASE, "ntire24_shrem_valid_inp"), os.path.join(LOCAL_BASE, "ntire24_shrem_valid_gt"), False)

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=2)

    model = fusion_net().to(device)

    # Losses
    l1_crit = nn.L1Loss().to(device)
    lpips_crit = lpips.LPIPS(net='vgg').to(device)
    fft_crit = FocalFrequencyLoss(loss_weight=0.1, alpha=1.0).to(device)
    msssim_crit = MultiScaleSSIMLoss(data_range=1.0).to(device)

    optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=1e-8)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, EPOCHS, eta_min=1e-7)

    # Resume Logic
    finetune_weight = os.path.join(DRIVE_BASE, "model_pro_epoch_5.pth")
    if os.path.exists(finetune_weight):
        model.load_state_dict(torch.load(finetune_weight, map_location=device), strict=False)
        print(f"✅ Loaded Best Weight: {finetune_weight}")
    else:
        print("⚠️ Warning: Previous weight not found. Starting fresh/base.")

    for epoch in range(EPOCHS):
        model.train()
        total_loss = 0

        print(f"\n--- Fine-Tune Epoch {epoch+1}/{EPOCHS} ---")
        for i, (inp, gt) in enumerate(train_loader):
            inp, gt = inp.to(device), gt.to(device)

            optimizer.zero_grad()
            out = model(inp)
            out_clamped = torch.clamp(out, 0, 1)

            loss = l1_crit(out, gt) + (0.05 * lpips_crit(out_clamped, gt).mean()) + fft_crit(out_clamped, gt) + (0.2 * (1 - msssim_crit(out_clamped, gt)))

            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            if i % 50 == 0:
                print(f"Iter {i}: Loss {loss.item():.4f}")

        scheduler.step()

        # Validation
        model.eval()
        psnr_list = []
        with torch.no_grad():
            for inp, gt in val_loader:
                inp, gt = inp.to(device), gt.to(device)
                out = torch.clamp(model(inp), 0, 1)
                mse = torch.mean((out - gt) ** 2).item()
                psnr = 100 if mse == 0 else 20 * np.log10(1.0 / np.sqrt(mse))
                psnr_list.append(psnr)

        avg_psnr = np.mean(psnr_list)
        print(f"🔴 EPOCH {epoch+1} DONE. Val PSNR: {avg_psnr:.4f} dB")

        torch.save(model.state_dict(), os.path.join(DRIVE_BASE, f"model_finetune_epoch_{epoch+1}.pth"))
        print(f"💾 Saved.")

if __name__ == "__main__":
    train()

some more changes in parameters trying to get better


In [ ]:
!pip install piq kornia lpips timm==0.6.13

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import os
import glob
import cv2
import numpy as np
import random
import sys
import shutil
import lpips
from piq import MultiScaleSSIMLoss

# --- 1. SETUP PATHS ---
%cd /content/FusionShadowRemoval-main
sys.path.append(os.getcwd())

DRIVE_BASE = "/content/drive/MyDrive/Shadow removal"
LOCAL_BASE = "/content/dataset"

# --- 2. FORCE DATA REPAIR (THE FIX) ---
def force_fix_data():
    print("🔄 Checking Dataset Health...")

    # Check if data exists AND is not empty
    inp_path = os.path.join(LOCAL_BASE, "ntire24_shrem_train_inp")

    # Agar folder nahi hai YA folder khaali hai -> Re-copy karo
    if not os.path.exists(inp_path) or len(os.listdir(inp_path)) == 0:
        print("⚠️ Data missing or empty! Re-copying from Drive...")

        # Purana kachra saaf karo
        if os.path.exists(LOCAL_BASE):
            shutil.rmtree(LOCAL_BASE)

        # Fresh Copy
        os.makedirs(LOCAL_BASE, exist_ok=True)
        for folder in ["ntire24_shrem_train_inp", "ntire24_shrem_train_gt", "ntire24_shrem_valid_inp", "ntire24_shrem_valid_gt"]:
            src = os.path.join(DRIVE_BASE, folder)
            dst = os.path.join(LOCAL_BASE, folder)
            if os.path.exists(src):
                print(f"   📂 Copying {folder}...")
                shutil.copytree(src, dst)
            else:
                print(f"❌ Error: {folder} Drive pe nahi mila!")
        print("✅ Data Restoration Complete!")
    else:
        print("✅ Data is present and ready.")

force_fix_data()


# --- 2. IMPORT MODEL ---
try:
    from models.model_convnext import fusion_net
    print("✅ Model imported!")
except ImportError:
    print("❌ Error: Model import fail hua.")
    sys.exit(1)

# --- 3. ADVANCED LOSSES ---
class FocalFrequencyLoss(nn.Module):
    def __init__(self, loss_weight=1.0, alpha=1.0):
        super().__init__()
        self.loss_weight = loss_weight
        self.alpha = alpha
    def forward(self, pred, target):
        return self.loss_weight * torch.mean(torch.abs(torch.fft.rfftn(pred) - torch.fft.rfftn(target)))

# --- 4. CONFIGURATION (SAFE & SLOW) ---
EPOCHS = 20
BATCH_SIZE = 1
PATCH_SIZE = 256
LR = 2e-6             # <-- VERY LOW LR (Safety Lock)

# --- 5. DATASET (NO AUGMENTATION - SAFE MODE) ---
class ShadowDataset(Dataset):
    def __init__(self, inp_dir, gt_dir, is_train=True):
        self.inp_files = sorted(glob.glob(os.path.join(inp_dir, "*")))
        self.gt_dir = gt_dir
        self.is_train = is_train
        self.valid_files = [f for f in self.inp_files if os.path.exists(os.path.join(gt_dir, os.path.basename(f)))]

    def __len__(self): return len(self.valid_files)

    def __getitem__(self, idx):
        inp_path = self.valid_files[idx]
        fname = os.path.basename(inp_path)
        gt_path = os.path.join(self.gt_dir, fname)
        img_inp = cv2.imread(inp_path)
        img_gt = cv2.imread(gt_path)

        if self.is_train:
            h, w = img_inp.shape[:2]
            if h > PATCH_SIZE and w > PATCH_SIZE:
                x = random.randint(0, w - PATCH_SIZE); y = random.randint(0, h - PATCH_SIZE)
                img_inp = img_inp[y:y+PATCH_SIZE, x:x+PATCH_SIZE]
                img_gt = img_gt[y:y+PATCH_SIZE, x:x+PATCH_SIZE]
            else:
                img_inp = cv2.resize(img_inp, (PATCH_SIZE, PATCH_SIZE))
                img_gt = cv2.resize(img_gt, (PATCH_SIZE, PATCH_SIZE))
        else:
            h, w = img_inp.shape[:2]
            new_h, new_w = ((h // 32) * 32), ((w // 32) * 32)
            img_inp = cv2.resize(img_inp, (new_w, new_h))
            img_gt = cv2.resize(img_gt, (new_w, new_h))

        inp_t = torch.from_numpy(img_inp[:, :, ::-1].copy()).permute(2, 0, 1).float() / 255.0
        gt_t = torch.from_numpy(img_gt[:, :, ::-1].copy()).permute(2, 0, 1).float() / 255.0
        return inp_t, gt_t

def train():
    device = torch.device('cuda:0')
    print(f"🚀 RELIABLE Fine-Tuning Started on {device}")

    if not os.path.exists(LOCAL_BASE):
        print("❌ Local Data missing. Run data copy first.")
        return

    train_ds = ShadowDataset(os.path.join(LOCAL_BASE, "ntire24_shrem_train_inp"), os.path.join(LOCAL_BASE, "ntire24_shrem_train_gt"), True)
    val_ds = ShadowDataset(os.path.join(LOCAL_BASE, "ntire24_shrem_valid_inp"), os.path.join(LOCAL_BASE, "ntire24_shrem_valid_gt"), False)

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=2)

    model = fusion_net().to(device)

    # Standard Losses (Jo pehle kaam kiye the)
    l1_crit = nn.L1Loss().to(device)
    lpips_crit = lpips.LPIPS(net='vgg').to(device)
    fft_crit = FocalFrequencyLoss(loss_weight=0.1, alpha=1.0).to(device)
    msssim_crit = MultiScaleSSIMLoss(data_range=1.0).to(device)

    optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=1e-8)

    # Scheduler
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, EPOCHS, eta_min=1e-7)

    # --- CRITICAL: LOAD THE GOOD WEIGHTS ---
    # Hum specific file load karenge jo humein pata hai achhi thi
    good_weight = os.path.join(DRIVE_BASE, "model_pro_epoch_5.pth")

    if os.path.exists(good_weight):
        print(f"✅ RECOVERING: Loading best model (26.1 dB): {good_weight}")
        model.load_state_dict(torch.load(good_weight, map_location=device), strict=False)
    else:
        print("⚠️ Warning: Good weight not found! Checking others...")
        # Fallback logic to avoid loading the bad "boost" model
        all_weights = sorted(glob.glob(os.path.join(DRIVE_BASE, "*.pth")), key=os.path.getmtime)
        valid_weights = [w for w in all_weights if "boost" not in w] # Exclude bad booster weights
        if valid_weights:
            print(f"✅ Loading: {valid_weights[-1]}")
            model.load_state_dict(torch.load(valid_weights[-1], map_location=device), strict=False)

    best_psnr = 26.0 # Baseline

    for epoch in range(EPOCHS):
        model.train()
        total_loss = 0

        print(f"\n--- Reliable Epoch {epoch+1}/{EPOCHS} ---")
        for i, (inp, gt) in enumerate(train_loader):
            inp, gt = inp.to(device), gt.to(device)

            optimizer.zero_grad()
            out = model(inp)
            out_clamped = torch.clamp(out, 0, 1)

            # Original Formula (No Greedy, No Risks)
            l1 = l1_crit(out, gt)
            perceptual = lpips_crit(out_clamped, gt).mean()
            fft = fft_crit(out_clamped, gt)
            ms_ssim = 1 - msssim_crit(out_clamped, gt)

            # Standard Weights
            loss = l1 + (0.05 * perceptual) + (0.1 * fft) + (0.2 * ms_ssim)

            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            if i % 50 == 0:
                print(f"Iter {i}: Loss {loss.item():.4f}")

        scheduler.step()

        # Validation
        model.eval()
        psnr_list = []
        with torch.no_grad():
            for inp, gt in val_loader:
                inp, gt = inp.to(device), gt.to(device)
                out = torch.clamp(model(inp), 0, 1)
                mse = torch.mean((out - gt) ** 2).item()
                psnr = 100 if mse == 0 else 20 * np.log10(1.0 / np.sqrt(mse))
                psnr_list.append(psnr)

        avg_psnr = np.mean(psnr_list)
        print(f"🔴 EPOCH {epoch+1} DONE. Val PSNR: {avg_psnr:.4f} dB")

        # Safe Name
        save_path = os.path.join(DRIVE_BASE, f"model_refined_epoch_{epoch+1}.pth")
        torch.save(model.state_dict(), save_path)
        print(f"💾 Saved: {save_path}")

        if avg_psnr > best_psnr:
            best_psnr = avg_psnr
            print("✨ Improvement detected!")

if __name__ == "__main__":
    train()

/content/FusionShadowRemoval-main
🔄 Checking Dataset Health...
⚠️ Data missing or empty! Re-copying from Drive...
   📂 Copying ntire24_shrem_train_inp...
   📂 Copying ntire24_shrem_train_gt...
   📂 Copying ntire24_shrem_valid_inp...
   📂 Copying ntire24_shrem_valid_gt...
✅ Data Restoration Complete!
✅ Model imported!
🚀 RELIABLE Fine-Tuning Started on cuda:0


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


100%|██████████| 528M/528M [00:07<00:00, 78.1MB/s]


Loading model from: /usr/local/lib/python3.12/dist-packages/lpips/weights/v0.1/vgg.pth
✅ RECOVERING: Loading best model (26.1 dB): /content/drive/MyDrive/Shadow removal/model_pro_epoch_5.pth

--- Reliable Epoch 1/20 ---
Iter 0: Loss 0.2190
Iter 50: Loss 0.2834
Iter 100: Loss 0.2658
Iter 150: Loss 0.2483
Iter 200: Loss 0.2226
Iter 250: Loss 0.2834
Iter 300: Loss 0.3640
Iter 350: Loss 0.2188
Iter 400: Loss 0.2929
Iter 450: Loss 0.2989
Iter 500: Loss 0.2341
Iter 550: Loss 0.2679
Iter 600: Loss 0.2691
Iter 650: Loss 0.2308
Iter 700: Loss 0.3192
Iter 750: Loss 0.2554
Iter 800: Loss 0.3314
Iter 850: Loss 0.2451
Iter 900: Loss 0.2609
Iter 950: Loss 0.2320
🔴 EPOCH 1 DONE. Val PSNR: 26.1406 dB
💾 Saved: /content/drive/MyDrive/Shadow removal/model_refined_epoch_1.pth
✨ Improvement detected!

--- Reliable Epoch 2/20 ---
Iter 0: Loss 0.2962
Iter 50: Loss 0.2909
Iter 100: Loss 0.3560
Iter 150: Loss 0.2162
Iter 200: Loss 0.2922
Iter 250: Loss 0.2348
Iter 300: Loss 0.2211
Iter 350: Loss 0.3314
Iter 40